In [24]:
import pandas as pd
import numpy as np

In [45]:
file = pd.read_csv('results/compared_bp_with_previous_year.csv')
# player_data = file

In [ ]:
##### import math
alphabet = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j']

player_data_df = pd.DataFrame({'RowId #':[0], 'PlayerId':[0], 'Value':[0], 'Property':[0], 'Team':[0]})
player_data_df.to_csv('results/concate_players.csv', ',')
    
for player, i in zip(file.playerID, range(len(file.playerID))): # For every player 624
    player_data = {}
    for col in file[file.playerID == player]: # Take all his data
        for item in file[file.playerID == player][col]: #
            for team in file[file.playerID == player].tmID:
                if item == item and not col == 'playerID': # is not NaN
                    item_str = ''
                    
                    if isinstance(item, int):
                        item = str(item)
                        for let in item:
                            item_str += alphabet[int(let)]
                        item = item_str
                    
                    if player_data != {}:
                        if not player in player_data['PlayerId'] and \
                        not item in player_data['Value'] and \
                        not col in player_data['Property'] and \
                        not team in player_data['Team']:
                            player_data.setdefault('RowId #', []).append('Row ' + str(i))
                            player_data.setdefault('PlayerId', []).append(player) #Sentence
                            player_data.setdefault('Value', []).append(item) #Word
                            player_data.setdefault('Property', []).append(col) #POS
                            player_data.setdefault('Team', []).append(team)  #Tag
                    else:
                            player_data.setdefault('RowId #', []).append('Row ' + str(i))
                            player_data.setdefault('PlayerId', []).append(player) #Sentence
                            player_data.setdefault('Value', []).append(item) #Word
                            player_data.setdefault('Property', []).append(col) #POS
                            player_data.setdefault('Team', []).append(team)  #Tag
    player_data_df = pd.DataFrame(player_data)
                    
    with open('results/concate_players.csv', 'a') as f:
        player_data_df.to_csv(f, header=False)

    del player_data_df

In [7]:
# player_data.to_csv('results/bap_bd_bpa_bp_bm.csv', ';', encoding='utf-8')

In [7]:
data = player_data.fillna(method="ffill")

In [10]:
words = list(set(data["Value"].values))

In [11]:
n_words = len(words); n_words

2298

In [12]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p, t) for w, p, t in zip(s["Value"].values.tolist(),
                                                           s["Property"].values.tolist(),
                                                           s["Team"].values.tolist())]
        self.grouped = self.data.groupby("RowId #").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped["Row {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None


In [13]:
getter = SentenceGetter(data)

In [14]:
sent = getter.get_next()

In [15]:
print(sent)

[('caaa', 'year', 'DEN'), ('caaa', 'year', 'DEN'), ('caaa', 'year', 'DAL'), ('caaa', 'year', 'DAL'), ('caab', 'year', 'DEN'), ('caab', 'year', 'DEN'), ('caab', 'year', 'DAL'), ('caab', 'year', 'DAL'), ('caab', 'year', 'DEN'), ('caab', 'year', 'DEN'), ('caab', 'year', 'DAL'), ('caab', 'year', 'DAL'), ('caac', 'year', 'DEN'), ('caac', 'year', 'DEN'), ('caac', 'year', 'DAL'), ('caac', 'year', 'DAL'), ('b', 'stint', 'DEN'), ('b', 'stint', 'DEN'), ('b', 'stint', 'DAL'), ('b', 'stint', 'DAL'), ('b', 'stint', 'DEN'), ('b', 'stint', 'DEN'), ('b', 'stint', 'DAL'), ('b', 'stint', 'DAL'), ('c', 'stint', 'DEN'), ('c', 'stint', 'DEN'), ('c', 'stint', 'DAL'), ('c', 'stint', 'DAL'), ('b', 'stint', 'DEN'), ('b', 'stint', 'DEN'), ('b', 'stint', 'DAL'), ('b', 'stint', 'DAL'), ('DEN', 'tmID', 'DEN'), ('DEN', 'tmID', 'DEN'), ('DEN', 'tmID', 'DAL'), ('DEN', 'tmID', 'DAL'), ('DEN', 'tmID', 'DEN'), ('DEN', 'tmID', 'DEN'), ('DEN', 'tmID', 'DAL'), ('DEN', 'tmID', 'DAL'), ('DAL', 'tmID', 'DEN'), ('DAL', 'tmID',

In [16]:
sentences = getter.sentences

In [17]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]

    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True

    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]

In [18]:
X = [sent2features(s) for s in sentences]
y = [sent2labels(s) for s in sentences]

In [19]:
from sklearn_crfsuite import CRF

crf = CRF(algorithm='lbfgs',
          c1=0.1,
          c2=0.1,
          max_iterations=100,
          all_possible_transitions=False)

In [20]:
from sklearn.cross_validation import cross_val_predict
from sklearn_crfsuite.metrics import flat_classification_report

/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [21]:
pred = cross_val_predict(estimator=crf, X=X, y=y, cv=5)

In [20]:
report = flat_classification_report(y_pred=pred, y_true=y)
print(report)

             precision    recall  f1-score   support

        ATL       0.36      0.02      0.03    136320
        BOS       0.52      0.06      0.11    147640
        CHA       0.22      0.01      0.01     31520
        CHH       0.30      0.01      0.03     34400
        CHI       0.30      0.03      0.06    113280
        CLE       0.55      0.02      0.03    126680
        DAL       0.69      0.05      0.09    123600
        DEN       0.50      0.03      0.05    130280
        DET       0.45      0.07      0.13    127760
        GSW       0.28      0.12      0.17     95280
        HOU       0.45      0.04      0.07    148160
        IND       0.04      0.99      0.08    121480
        LAC       0.66      0.04      0.08    117760
        LAL       0.46      0.17      0.24     95400
        MEM       0.50      0.11      0.18     88720
        MIA       0.54      0.02      0.04    103240
        MIL       0.42      0.05      0.09    125960
        MIN       0.55      0.07      0.12   

In [21]:
crf.fit(X, y)

CRF(algorithm='lbfgs', all_possible_states=None,
  all_possible_transitions=False, averaging=None, c=None, c1=0.1, c2=0.1,
  calibration_candidates=None, calibration_eta=None,
  calibration_max_trials=None, calibration_rate=None,
  calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
  gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
  max_linesearch=None, min_freq=None, model_filename=None,
  num_memories=None, pa_type=None, period=None, trainer_cls=None,
  variance=None, verbose=False)

In [22]:
import eli5

In [23]:
eli5.show_weights(crf, top=30)

From \ To,ATL,BOS,CHA,CHH,CHI,CLE,DAL,DEN,DET,GSW,HOU,IND,LAC,LAL,MEM,MIA,MIL,MIN,NJN,NOH,NOK,NYK,ORL,PHI,PHO,POR,SAC,SAS,SEA,TOR,UTA,VAN,WAS
ATL,4.419,1.947,0.0,0.0,0.816,1.693,1.869,0.0,2.78,0.0,1.519,2.094,1.472,2.014,3.052,1.858,1.084,1.858,1.571,0.0,0.0,2.102,2.249,1.468,0.964,2.892,0.0,0.0,1.927,2.165,0.0,0.0,0.492
BOS,3.419,5.638,0.0,4.16,0.0,2.731,3.343,3.028,2.723,0.0,0.0,2.12,1.244,3.902,0.0,3.239,0.0,3.98,1.854,0.0,0.0,2.678,-1.142,0.0,2.704,0.0,0.0,0.0,3.352,0.0,0.0,2.323,0.0
CHA,0.0,0.0,6.491,0.0,5.156,3.097,0.0,0.0,0.0,0.0,1.051,3.153,1.192,3.433,1.689,3.59,3.496,0.0,1.107,0.0,0.0,0.0,0.047,2.094,0.0,1.929,4.264,0.0,0.0,0.0,0.0,0.0,3.362
CHH,0.0,0.0,0.0,7.896,4.387,0.0,0.0,0.0,0.0,0.0,1.504,0.0,2.249,0.0,0.0,0.0,0.0,0.0,3.724,7.818,0.0,4.679,0.0,3.259,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CHI,-0.491,0.0,0.0,3.084,5.964,-0.52,1.687,0.0,0.783,2.439,0.0,4.012,2.328,2.367,0.0,2.612,2.143,1.972,-1.063,0.0,0.0,3.163,0.0,0.473,2.378,1.797,1.783,1.157,0.0,3.886,0.0,4.523,1.172
CLE,2.977,3.59,1.81,2.741,0.0,4.133,1.12,-0.018,1.11,2.332,-0.75,0.0,0.765,0.0,3.275,2.656,2.343,0.0,2.555,3.535,0.0,2.454,1.491,2.854,0.0,2.119,0.724,1.785,2.778,2.318,3.144,3.976,0.0
DAL,2.992,2.233,0.0,0.0,0.0,1.462,4.758,2.644,1.854,3.353,0.814,0.0,1.059,0.0,0.0,0.0,1.814,1.899,1.814,2.371,0.0,1.64,1.25,0.393,0.881,0.0,2.237,2.057,2.893,0.0,2.167,0.0,2.826
DEN,1.856,2.074,0.0,0.0,0.0,1.339,3.201,4.51,0.0,2.183,1.571,0.769,0.322,-1.121,0.0,1.561,0.097,0.709,0.843,1.998,0.0,1.496,2.275,-1.297,0.569,1.523,0.0,2.531,2.709,2.604,3.426,0.0,0.0
DET,2.356,3.026,0.0,3.854,0.0,0.0,0.0,2.634,5.762,2.488,1.231,1.29,0.0,0.0,0.0,-0.856,3.032,2.987,1.892,0.0,0.0,0.0,2.154,1.245,2.229,3.044,2.849,0.0,0.0,2.616,2.205,0.0,3.06
GSW,0.0,1.616,0.0,3.474,0.382,-0.333,2.63,2.593,2.505,5.561,-1.382,2.371,0.0,2.19,2.449,-1.015,1.011,3.185,1.318,2.772,0.0,0.0,0.0,0.893,0.996,2.034,0.0,0.941,2.457,0.0,-0.234,0.0,2.869


In [ ]:
crf = CRF(algorithm='lbfgs',
          c1=10,
          c2=0.1,
          max_iterations=100,
          all_possible_transitions=False)

In [ ]:
pred = cross_val_predict(estimator=crf, X=X, y=y, cv=5)

In [ ]:
report = flat_classification_report(y_pred=pred, y_true=y)
print(report)

In [ ]:
crf.fit(X, y)

In [ ]:
eli5.show_weights(crf, top=30)